# 🤖 Using LLMs with LangChain - A Beginner's Guide

Welcome to this interactive tutorial! In this notebook, you'll learn how to:
- 🔧 Set up and configure chat models
- 💬 Interact with AI using different message types
- 🚀 Make your first API calls to language models

Let's dive in! 🎯

---

## 📚 What are Chat Models?

Chat models are AI systems that:
- Take a **sequence of messages** as input
- Return **intelligent responses** as output

Think of it like having a conversation with an AI assistant!

### 🌟 Why LangChain?

LangChain makes it easy to work with different AI models through a unified interface. In this course, we'll use:

| Model | Why? | Link |
|-------|------|------|
| **ChatOpenAI** | Popular & performant | [Documentation](https://docs.langchain.com/oss/python/integrations/chat/openai) |

> **📝 Note:** You'll need an `OPENAI_API_KEY` to follow along. Don't have one? Check out [OpenAI's platform](https://platform.openai.com/).

---

## 🔑 Step 1: Environment Setup

First, let's load our API keys from the environment. This keeps your secrets safe! 🔒

In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✅ Environment variables loaded successfully!")

---

## 🎛️ Step 2: Initialize the Chat Model

Now we'll set up our AI model. We have two options:

### Option A: Using OpenRouter 🔄

OpenRouter allows you to access multiple AI models through a single API.

In [ ]:
from langchain_openai import ChatOpenAI
import os

# Configure model with OpenRouter
model = ChatOpenAI(
    model="gpt-4.1-mini",  # Specify the model you want to use
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=0,  # 0 = deterministic, 1 = creative
)

print("🎉 Model initialized with OpenRouter!")

### Option B: Direct OpenAI Connection 🎯

Or connect directly to OpenAI's API:

In [ ]:
from langchain_openai import ChatOpenAI

# Simple direct connection
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

print("🎉 Model initialized with OpenAI!")

#### 🌡️ Understanding Temperature

The `temperature` parameter controls response creativity:

```
0.0 ──────────────────────── 1.0
│                            │
Predictable              Creative
Consistent               Varied
Focused                  Exploratory
```

- **temperature=0**: Best for factual answers, code, translations
- **temperature=0.7**: Good balance for most tasks
- **temperature=1**: Maximum creativity for stories, brainstorming

---

## 🚀 Step 3: Your First AI Interaction!

Let's send a message to the AI and see what happens! 🎊

In [ ]:
# Sending a simple string message
response = model.invoke("Hello, world!")

print("📩 Full Response Object:")
print("=" * 50)
print(response)
print("=" * 50)

### 🔍 Understanding the Response

The response object contains:
- `content`: The actual AI message
- `response_metadata`: Token usage, costs, model info
- `id`: Unique identifier for this interaction

Most of the time, you'll just want the content:

In [ ]:
# Extract just the message content
print("💬 AI says:")
print(response.content)

---

## 💬 Step 4: Working with Message Roles

In real conversations, different participants have different roles. In AI chat, we typically have:

| Role | Purpose | Example |
|------|---------|------|
| **System** 🎭 | Sets behavior/personality | "You are a helpful coding assistant" |
| **User** 👤 | Your questions/inputs | "What is the capital of France?" |
| **Assistant** 🤖 | AI's responses | "The capital of France is Paris." |

![Message Flow](https://via.placeholder.com/700x200/9B59B6/FFFFFF?text=System+Sets+Rules+→+User+Asks+→+Assistant+Answers)

### 🎯 System Messages: Setting the Stage

System messages are like giving the AI a role to play:

In [ ]:
# Creating a structured conversation
response = model.invoke(
    [
        {
            "role": "system",
            "content": "You are a helpful geography expert who loves to share interesting facts.",
        },
        {"role": "user", "content": "What is the capital of France?"},
    ]
)

print("🌍 Geography Expert says:")
print(response.content)

---

## 📨 Step 5: Using Message Objects

LangChain provides special message classes for cleaner code:

```python
HumanMessage    → Messages from the user
AIMessage       → Responses from the AI  
SystemMessage   → System instructions
```

### 🎨 Benefits of Message Objects:
- 📝 More readable code
- 🏷️ Can add metadata (like names)
- 🔄 Easier to build conversation histories

In [ ]:
from langchain_core.messages import HumanMessage

# Create a message with metadata
msg = HumanMessage(
    content="Hello world! I'm excited to learn about AI!",
    name="Student",  # Optional: identify the speaker
)

# Send as a list
messages = [msg]
response = model.invoke(messages)

print("🎓 AI's response to student:")
print(response.content)

The response contains multiple pieces of information:

```python
AIMessage(
    content='...',              # The actual message text
    response_metadata={         # Metadata about the response
        'token_usage': {...},   # How many tokens were used
        'model_name': '...',    # Which model processed this
        'finish_reason': '...'  # Why the response ended
    },
    id='...',                   # Unique identifier
)
```

### ⚡ Quick Tip: Shorthand Method

For simple queries, you can skip the message objects entirely:

In [ ]:
# This automatically converts to a HumanMessage
response = model.invoke("What's the weather like?")
print(response.content)

---

## 🛠️ Step 6: Essential Methods

LangChain chat models come with powerful methods:

 📞 `invoke()` - Single Response
```python
response = model.invoke("Your question")
# Returns complete response at once
```

**Best for:** Short answers, quick questions

---

🌊 `stream()` - Real-time Streaming
```python
for chunk in model.stream("Tell me a story"):
    print(chunk.content, end="")
# Shows response as it's generated
```

**Best for:** Long responses, better UX

In [ ]:
for chunk in model.stream("Tell me a story"):
    print(chunk.content, end="")

Sometimes you might want to use the builtin rompttemplate and the chains of LangChains (But I don't really like them :p)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

You can use tuple for messages

In [ ]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet."
            "Always provide detailed recommendations, including requests for length, virality, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Or you can use dicts

In [ ]:
generation_prompt = ChatPromptTemplate.from_messages(
    [
        {
            "role": "system",
            "content": "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            " Generate the best twitter post possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        },
        {
            "role": "user",
            "content": "I want to write a twitter post about the latest trends in AI.",
        },
    ]
)

In [ ]:
# Using message objects with metadata
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="You are a helpful coding tutor specializing in Python."),
    HumanMessage(
        content="How do I reverse a string in Python?",
        name="Student",  # Optional: identify the speaker
    ),
]

response = model.invoke(messages)

print("🎓 Tutor's Response:")
print(response.content)

### Building Conversation History 💭

For multi-turn conversations, include previous messages:

In [ ]:
# Simulating a multi-turn conversation
conversation = [
    SystemMessage(content="You are a math tutor."),
    HumanMessage(content="What is 15 * 24?"),
    AIMessage(content="15 * 24 = 360"),
    HumanMessage(content="Can you show me how you calculated that?"),
]

response = model.invoke(conversation)

print("📚 Math Tutor:")
print(response.content)


## LangChain Expression Language (LCEL) 🔗

### What is LCEL? 🤔

LCEL is LangChain's way of chaining components together using the `|` (pipe) operator.

```python
# Traditional approach
formatted = prompt.format(...)
response = model.invoke(formatted)

# LCEL approach ✨
chain = prompt | model
response = chain.invoke({...})
```

### Why Use LCEL? 🌟

| Feature | Benefit |
|---------|--------|
| **Composability** | Chain multiple operations easily |
| **Streaming** | Automatic streaming support |
| **Async** | Built-in async execution |
| **Batching** | Process multiple inputs efficiently |
| **Debugging** | Better error messages and tracing |

### Visual Representation

```
Input ──> Prompt ──> Model ──> Output Parser ──> Final Result
          Template
          
With LCEL:
chain = prompt | model | output_parser
result = chain.invoke(input)
```

In [ ]:
llm = ChatOpenAI(model="o4-mini")

generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

In [ ]:
generate_chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="I want to write a twitter post about the latest trends in AI."
            )
        ]
    }
)

In [ ]:
reflect_chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="I want to write a twitter post about the latest trends in AI."
            )
        ]
    }
)

## 📝 Summary

Congratulations! 🎉 You've learned:

✅ How to set up LangChain chat models  
✅ The difference between message roles  
✅ How to invoke models with strings and message objects  
✅ Understanding response objects and extracting content  

### 🚀 Next Steps:

- Learn about prompt templates
- Build conversation chains
- Explore memory and context
- Create RAG agents

### 📚 Resources:

- [LangChain Documentation](https://docs.langchain.com)
- [LangChain Python Reference](https://reference.langchain.com/python)
- [OpenAI API Documentation](https://platform.openai.com/docs)

---

**Happy coding!** 🚀✨